In [484]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pkl
import spacy
from nltk.stem.porter import PorterStemmer

Data Load

In [420]:
movies=pd.read_csv('../Datasets/tmdb_5000_movies.csv')
credits=pd.read_csv('../Datasets/tmdb_5000_credits.csv')

In [501]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [422]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [423]:
movies=movies.merge(credits,on='title')

In [424]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [505]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [426]:
# Useful columns
# genres, id, keywords,overview,title,cast, crew
movies=movies[['genres','movie_id', 'keywords','overview','title','cast', 'crew']]

In [427]:
movies.head(1)

,genres,movie_id,keywords,overview,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [428]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4809 non-null   object
 1   movie_id  4809 non-null   int64 
 2   keywords  4809 non-null   object
 3   overview  4806 non-null   object
 4   title     4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [429]:
movies.isna().sum()

genres      0
movie_id    0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [430]:
movies[movies['overview'].isna()]

,genres,movie_id,keywords,overview,title,cast,crew
2658,"[{""id"": 18, ""name"": ""Drama""}]",370980,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",NaN,Chiamatemi Francesco - Il Papa della gente,"[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4145,"[{""id"": 99, ""name"": ""Documentary""}]",459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",NaN,"To Be Frank, Sinatra at 100","[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4437,"[{""id"": 99, ""name"": ""Documentary""}]",292539,[],NaN,Food Chains,[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [511]:
print(movies[movies['title'] == 'Avatar']['overview'][0])

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.


In [432]:
movies.loc[movies['title'] == 'Food Chains', 'overview'] = """A look into the harsh reality of farm labor in America, where poor wages and exploitation persist, driven by powerful supermarket chains at the top of the food industry."""

In [433]:
movies.loc[movies['title'] == 'To Be Frank, Sinatra at 100', 'overview'] = """A tribute to the legendary Frank Sinatra, celebrating his 100th birthday with rare footage, interviews, and a look at the lasting impact of his music and persona."""

In [434]:
movies.loc[movies['title'] == 'Chiamatemi Francesco - Il Papa della gente', 'overview'] = """The inspiring journey of Jorge Mario Bergoglio, from his humble beginnings in Argentina to becoming Pope Francis, a voice for the poor and a symbol of compassion and humility."""

In [ ]:
# movies.dropna(inplace=True)

In [435]:
movies.isna().sum()

genres      0
movie_id    0
keywords    0
overview    0
title       0
cast        0
crew        0
dtype: int64

In [436]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

- '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
- we have to change it all like this:  ['Action','Adventure','Fantasy','SciFi']


In [437]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
    

In [438]:
movies['genres']=movies['genres'].apply(convert)


In [439]:
movies['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4809, dtype: object

In [440]:
movies['keywords']

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4804    [{"id": 5616, "name": "united states\u2013mexi...
4805                                                   []
4806    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4807                                                   []
4808    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4809, dtype: object

In [441]:
movies['keywords']=movies['keywords'].apply(convert)

In [442]:
movies['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4809, dtype: object

In [443]:
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [ ]:
def convert_cast(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l
    

In [445]:
movies['cast']=movies['cast'].apply(convert_cast)

In [526]:
movies['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4809, dtype: object

In [447]:
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [449]:
movies['crew']=movies['crew'].apply(fetch_director)

In [450]:
movies['crew'][0]

['James Cameron']

In [451]:
movies.head()

,genres,movie_id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [452]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [453]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [454]:
movies.head()

,genres,movie_id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [ ]:
# tranform name having space to no spaced name
movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ",'') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ",'') for i in x])
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ",'') for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ",'') for i in x])

In [458]:
movies.head()

,genres,movie_id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [495]:
movies['overview']=movies['overview'].apply(lambda x:" ".join(x))
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
# new_df['tags']=new_df['tags'].apply(lambda x : x.lower())
# from nltk.stem.porter import PorterStemmer
# ps=PorterStemmer()
# def stem(text):
#     y=[]
#     for i in text.split():
#        y.append(ps.stem(i))
#     return ' '.join(y)

#### 🧪 Preprocessing Strategy

We apply **SpaCy lemmatization** only to the `overview` column, because:

- Overview contains full sentences with grammar, stop words, and punctuation.
- Lemmatization improves consistency (e.g., "running", "runs" → "run") for better semantic matching.

We do **not lemmatize the `tags`** column because:
- It is already structured, made from genres, cast, director, and keywords.
- Lemmatizing proper nouns (like names) could corrupt the original data.

This selective preprocessing makes our vectorization cleaner and more meaningful.


In [496]:
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Apply only to overview
movies['overview'] = movies['overview'].fillna("").apply(lemmatize_text)

In [ ]:
movies['tags']=movies['cast']+movies['crew']+movies['genres']+movies['keywords']

In [540]:
movies.head()

,genres,movie_id,keywords,overview,title,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",22nd century paraplegic Marine dispatch moon P...,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Captain Barbossa long believe dead come life h...,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",cryptic message Bond past send trail uncover s...,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",follow death District Attorney Harvey Dent Bat...,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter war weary military captain inexpli...,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [461]:
movies['tags'][0]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [ ]:
new_df=movies[['movie_id','title','overview','tags']]

In [463]:
new_df.head()

,movie_id,title,overview,tags
0,19995,Avatar,22nd century paraplegic Marine dispatch moon P...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,Captain Barbossa long believe dead come life h...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,Spectre,cryptic message Bond past send trail uncover s...,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,The Dark Knight Rises,follow death District Attorney Harvey Dent Bat...,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,John Carter,John Carter war weary military captain inexpli...,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [464]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_19236\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [465]:
new_df['tags'][0]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [466]:
new_df['tags']=new_df['tags'].apply(lambda x : x.lower())

C:\Users\Admin\AppData\Local\Temp\ipykernel_19236\3923394162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x : x.lower())


In [467]:
new_df['tags'][0]

'samworthington zoesaldana sigourneyweaver jamescameron action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [468]:
new_df.head()

,movie_id,title,overview,tags
0,19995,Avatar,22nd century paraplegic Marine dispatch moon P...,samworthington zoesaldana sigourneyweaver jame...
1,285,Pirates of the Caribbean: At World's End,Captain Barbossa long believe dead come life h...,johnnydepp orlandobloom keiraknightley gorever...
2,206647,Spectre,cryptic message Bond past send trail uncover s...,danielcraig christophwaltz léaseydoux sammende...
3,49026,The Dark Knight Rises,follow death District Attorney Harvey Dent Bat...,christianbale michaelcaine garyoldman christop...
4,49529,John Carter,John Carter war weary military captain inexpli...,taylorkitsch lynncollins samanthamorton andrew...


In [485]:
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text.lower())  # Convert to lowercase and process with spaCy
    tokens = []
    for token in doc:
        # Remove punctuation, stop words, and non-alphabetic tokens
        if not token.is_stop and not token.is_punct and token.is_alpha:
            tokens.append(token.lemma_)  # Lemmatize the word
    return " ".join(tokens)


In [486]:
new_df['tags']=new_df['tags'].apply(preprocess)

KeyboardInterrupt: 

#### Why I Chose Lemmatization over Stemming

I'm working with user inputs and textual tags like _"fighting"_, _"fighter"_, _"fights"_, etc. I want all of these variations to be grouped under a single base word — **"fight"** — so that the similarity matching works more meaningfully.

Lemmatization helps me achieve this because it preserves **actual dictionary words**, unlike stemming which often cuts words awkwardly (like _"romance"_ becoming _"romanc"_).

By using lemmatization, I ensure that my model performs smarter and more accurate similarity comparisons, especially when using **TF-IDF** or **CountVectorizer**. This helps my Movie Recommendation System understand the context better and provide more relevant results.


In [487]:
new_df['tags'][0]

'samworthington zoesaldana sigourneyweaver jamescameron action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [472]:
# cv=CountVectorizer(max_features=5000,stop_words='english')


In [473]:
# vectors = cv.fit_transform(new_df['tags']).toarray()

In [474]:
# vectors.shape

(4809, 5000)

In [475]:
# cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [476]:
similarity = cosine_similarity(final_vectors)
similarity.shape

(4809, 4809)

(4809, 4809)

In [478]:
def recommed(movie):
    movi_index=new_df[new_df['title']==movie].index[0]
    distance=similarity[movi_index]

    movies_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
       
    

In [479]:
recommed('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [ ]:
recommed("Avatar")

Star Trek Into Darkness
The Lovers
Jupiter Ascending
Aliens
Megaforce


In [480]:
pkl.dump(new_df.to_dict(),open('../Artifacts/movies_dict.pkl','wb'))

In [481]:
pkl.dump(similarity,open('../Artifacts/similsrity.pkl','wb'))